# Face Detection, Face Analysis, Face Recognition

Requirements: Azure AI Face Resource

Face detection and analysis: Detect human faces in an image and return the rectangle coordinates of their locations, and optionally with landmarks, and face-related attributes. This operation is required as a first step in all the other face recognition scenarios.

Face recognition: Confirm that a user is who they claim to be based on how closely their face data matches the target face. It includes Face verification ("one-to-one" matching).

Business Case:
Building Security where employee faces are saved into the system

Determine Input Requirements:
https://learn.microsoft.com/en-us/azure/ai-services/computer-vision/overview-identity#input-requirements

## Install Library

In [ ]:
%pip install --upgrade azure-ai-vision-face

## Load Azure Configuration

In [ ]:
import os
azure_face_endpoint = os.environ["AZURE_FACE_ENDPOINT"]
azure_face_key = os.environ["AZURE_FACE_KEY"]

## Create Clients

https://learn.microsoft.com/en-us/python/api/overview/azure/ai-vision-face-readme?view=azure-python-preview#key-concepts

In [ ]:
import uuid

from azure.core.credentials import AzureKeyCredential
from azure.ai.vision.face import FaceAdministrationClient, FaceClient
from azure.ai.vision.face.models import FaceAttributeTypeRecognition04, FaceDetectionModel, FaceRecognitionModel, QualityForRecognition,  FaceAttributeTypeDetection03

face_admin_client = FaceAdministrationClient(endpoint=azure_face_endpoint, credential=AzureKeyCredential(azure_face_key))
face_client = FaceClient(endpoint=azure_face_endpoint, credential=AzureKeyCredential(azure_face_key))

## Helper Function

In [ ]:
# Returns the file content of the image in the specified path
def read_file_content(file_path: str):
    with open(file_path, "rb") as fd:
        file_content = fd.read()
    return file_content

## Detect and Analyze Faces

Detects the face and its attributes 

Detection and Recognition Model Definitions: https://learn.microsoft.com/en-us/python/api/azure-ai-vision-face/azure.ai.vision.face.faceclient?view=azure-python-preview#azure-ai-vision-face-faceclient-detect

Face Attributes: https://learn.microsoft.com/en-us/python/api/azure-ai-vision-face/azure.ai.vision.face.models.faceattributetype?view=azure-python-preview


In [ ]:
sample_file_path = "../data/face/group/group_1.jpeg"

file_content = read_file_content(sample_file_path)

result = face_client.detect(
    file_content,
    detection_model=FaceDetectionModel.DETECTION03,  # The latest detection model.
    recognition_model=FaceRecognitionModel.RECOGNITION04,  # The latest recognition model.
    return_face_id=True,
    return_face_attributes=[
        FaceAttributeTypeDetection03.HEAD_POSE,
        FaceAttributeTypeDetection03.MASK,
        FaceAttributeTypeRecognition04.QUALITY_FOR_RECOGNITION,
    ],
    return_face_landmarks=True,
    return_recognition_model=True,
    face_id_time_to_live=120,
)

print(f"Detect faces from the file: {sample_file_path}")
# Print all detection results per line
for face in result:
    print(f"Face ID: {face.face_id}")
    print(f"Face Rectangle: {face.face_rectangle.as_dict()}")
    print(f"Face Landmarks:")
    for landmark, position in face.face_landmarks.as_dict().items():
        print(f"  {landmark}: {position}")
    print(f"Face Attributes: {face.face_attributes.as_dict()}")
    print(f"Recognition Model: {face.recognition_model}")
    print()

## Show the face bounding box and the landmark points

In [ ]:
from PIL import Image, ImageDraw

image = Image.open(sample_file_path)
draw = ImageDraw.Draw(image)

for idx, face in enumerate(result):
    # Draw bounding box
    rect = face.face_rectangle
    left = rect.left
    top = rect.top
    right = left + rect.width
    bottom = top + rect.height
    draw.rectangle([left, top, right, bottom], outline="red", width=3)
    
    # Draw landmarks
    for landmark in face.face_landmarks.as_dict().values():
        x, y = landmark['x'], landmark['y']
        draw.ellipse((x-2, y-2, x+2, y+2), fill="blue")

# Save or display the image with bounding boxes and landmarks
#output_path = "../Data/images/group/group_1_with_boxes_and_landmarks.jpg"
#image.save(output_path)
image.show()

## Create Large Person Group

In [ ]:
# Used in the Large Person Group Operations and Delete Large Person Group examples.
# LARGE_PERSON_GROUP_ID should be all lowercase and alphanumeric. For example, 'mygroupname' (dashes are OK).
LARGE_PERSON_GROUP_ID = str(uuid.uuid4())  # assign a random ID (or name it anything)

# Create a Large Person Group
print("Person group:", LARGE_PERSON_GROUP_ID)
face_admin_client.large_person_group.create(
    large_person_group_id=LARGE_PERSON_GROUP_ID,
    name=LARGE_PERSON_GROUP_ID,
    recognition_model=FaceRecognitionModel.RECOGNITION04,
)

## Add Persons to Group

In [ ]:
# Define person
person_A = face_admin_client.large_person_group.create_person(
    large_person_group_id=LARGE_PERSON_GROUP_ID,
    name="Person A",
)

# Images
person_A_image_file_path = [
    "../data/face/person_a/person_a1.jpeg",
    "../data/face/person_a/person_a2.jpeg",
]

for image in person_A_image_file_path:
    # Read the image file
    file_content = read_file_content(image)

    # Detect the face 
    result = face_client.detect(
        file_content,
        detection_model=FaceDetectionModel.DETECTION03,  # The latest detection model.
        recognition_model=FaceRecognitionModel.RECOGNITION04,  # The latest recognition model.
        return_face_id=True,
        return_face_attributes=[FaceAttributeTypeRecognition04.QUALITY_FOR_RECOGNITION]
    )

    # Ensure only one face is detected and it has high quality for recognition
    if len(result) == 1 and result[0].face_attributes.quality_for_recognition == QualityForRecognition.HIGH:
        face_admin_client.large_person_group.add_face(
            LARGE_PERSON_GROUP_ID,
            person_A.person_id,
            file_content,
            detection_model=FaceDetectionModel.DETECTION03,        
    )
        
    print(f"Face {result[0].face_id} added to person {person_A.person_id}")

In [ ]:
# Define person
person_B = face_admin_client.large_person_group.create_person(
    large_person_group_id=LARGE_PERSON_GROUP_ID,
    name="Person B",
)

# Images
person_B_image_file_path = [
    "../data/face/person_b/person_b1.jpeg",
    "../data/face/person_b/person_b2.jpeg",
]

for image in person_B_image_file_path:
    # Read the image file
    file_content = read_file_content(image)

    # Detect the face 
    result = face_client.detect(
        file_content,
        detection_model=FaceDetectionModel.DETECTION03,  # The latest detection model.
        recognition_model=FaceRecognitionModel.RECOGNITION04,  # The latest recognition model.
        return_face_id=True,
        return_face_attributes=[FaceAttributeTypeRecognition04.QUALITY_FOR_RECOGNITION]
    )

    # Ensure only one face is detected and it has high quality for recognition
    if len(result) == 1 and result[0].face_attributes.quality_for_recognition == QualityForRecognition.HIGH:
        face_admin_client.large_person_group.add_face(
            LARGE_PERSON_GROUP_ID,
            person_B.person_id,
            file_content,
            detection_model=FaceDetectionModel.DETECTION03,        
    )
        
    print(f"Face {result[0].face_id} added to person {person_B.person_id}")

## Train the Large Person Group

In [ ]:
# Train the large person group and set the polling interval to 5s
print(f"Train the person group {LARGE_PERSON_GROUP_ID}")
poller = face_admin_client.large_person_group.begin_train(
    large_person_group_id=LARGE_PERSON_GROUP_ID,
    polling_interval=5,
)

poller.wait()
print(f"The person group {LARGE_PERSON_GROUP_ID} is trained successfully.")

## Identify Faces in NEW Images and Determine their Identity in Large Person Group

In [ ]:
# Image for testing against
image="../data/face/group/group_2.jpeg"
file_content = read_file_content(image)

# Face IDs collection for all detected faces that are of sufficient quality for recognition
face_ids = []

# We use detection model 03 to get better performance, recognition model 04 to support quality for recognition attribute.
# Check if the image is of sufficient quality for recognition.
faces = face_client.detect(
    file_content,
    detection_model=FaceDetectionModel.DETECTION03,  # The latest detection model.
    recognition_model=FaceRecognitionModel.RECOGNITION04,  # The latest recognition model.
    return_face_id=True,
    return_face_attributes=[FaceAttributeTypeRecognition04.QUALITY_FOR_RECOGNITION]
)

for face in faces:
    # Only take the face if it is of sufficient quality.
    if face.face_attributes.quality_for_recognition != QualityForRecognition.LOW:
        face_ids.append(face.face_id)

# Identify faces
identify_results = face_client.identify_from_large_person_group(
    face_ids=face_ids,
    large_person_group_id=LARGE_PERSON_GROUP_ID,
)
print("Identifying faces in image")
for identify_result in identify_results:
    if identify_result.candidates:
        print(f"Person is identified for face ID {identify_result.face_id} in image, with a confidence of "
              f"{identify_result.candidates[0].confidence}.")  # Get topmost confidence score

    # Verify faces
        verify_result = face_client.verify_from_large_person_group(
            face_id=identify_result.face_id,
            large_person_group_id=LARGE_PERSON_GROUP_ID,
            person_id=identify_result.candidates[0].person_id,
        )
        print(f"Person Identified: {identify_result.candidates[0].person_id}. Verification result: {verify_result.is_identical}. confidence: {verify_result.confidence}")
    else:
        print(f"No person identified for face ID {identify_result.face_id} in image.")


## [Optional] Identify Faces in Images and place bounding boxes

In [ ]:
from PIL import Image, ImageDraw, ImageFont

# Group image for testing against
image_path = "../data/face/group/group_2.jpeg"
file_content = read_file_content(image_path)

# Detect faces
face_ids = []
face_rectangles = {}
# We use detection model 03 to get better performance, recognition model 04 to support quality for recognition attribute.
# Check if the image is of sufficient quality for recognition.
faces = face_client.detect(
    file_content,
    detection_model=FaceDetectionModel.DETECTION03,  # The latest detection model.
    recognition_model=FaceRecognitionModel.RECOGNITION04,  # The latest recognition model.
    return_face_id=True,
    return_face_attributes=[FaceAttributeTypeRecognition04.QUALITY_FOR_RECOGNITION]
)

# Open the image using PIL
image = Image.open(image_path)
draw = ImageDraw.Draw(image)
font = ImageFont.load_default()

for face in faces:
    # Only take the face if it is of sufficient quality.
    if face.face_attributes.quality_for_recognition != QualityForRecognition.LOW:
        face_ids.append(face.face_id)
        # Store face rectangle for later use
        face_rectangles[face.face_id] = face.face_rectangle
        # Draw bounding box
        rect = face.face_rectangle
        left = rect.left
        top = rect.top
        right = left + rect.width
        bottom = top + rect.height
        draw.rectangle(((left, top), (right, bottom)), outline="red", width=2)

# Identify faces
identify_results = face_client.identify_from_large_person_group(
    face_ids=face_ids,
    large_person_group_id=LARGE_PERSON_GROUP_ID,
)
print("Identifying faces in image")
for identify_result in identify_results:
    if identify_result.candidates:
        person_id = identify_result.candidates[0].person_id
        confidence = identify_result.candidates[0].confidence
        print(f"Person is identified for face ID {identify_result.face_id} in image, with a confidence of {confidence}.")  # Get topmost confidence score

        # Verify faces
        verify_result = face_client.verify_from_large_person_group(
            face_id=identify_result.face_id,
            large_person_group_id=LARGE_PERSON_GROUP_ID,
            person_id=person_id,
        )
        print(f"Person Identified: {person_id}. Verification result: {verify_result.is_identical}. confidence: {verify_result.confidence}")

        # Draw person ID and confidence on the image
        rect = face_rectangles[identify_result.face_id]
        left = rect.left
        top = rect.top
        draw.text((left, top - 10), f"ID: {person_id}, Conf: {confidence:.2f}", fill="red", font=font)
    else:
        print(f"No person identified for face ID {identify_result.face_id} in image.")

# Save or display the image with bounding boxes and labels
image.show()  # To display the image
#image.save("output_image_with_bounding_boxes_and_labels.jpg")  # To save the image

## Clean up resources

In [ ]:
# Delete the large person group
face_admin_client.large_person_group.delete(LARGE_PERSON_GROUP_ID)
print(f"The person group {LARGE_PERSON_GROUP_ID} is deleted.")